In [1]:
import pandas as pd, numpy as np, os, sys

In [2]:
df_trans = pd.read_csv('../accounts/transactions(2).csv', parse_dates=['Book date'])
df_trans.drop(columns=['Account number(BBAN)', 'Account name', 'Account number(IBAN)', 'Description',
                       'Transaction reference', 'Instructed amount', 'Transaction type', 'Transaction group',
                       'Instructed currency', 'currency exchange rate', 'Instructed amount', 'Category',
                       'Check number', 'Creation Time'], inplace=True)

/var/folders/r7/w0dhw84s7kd0n75_1lp51j300000gn/T/ipykernel_7651/3822365868.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df_trans = pd.read_csv('../accounts/transactions(2).csv', parse_dates=['Book date'])


In [3]:
# import pandas as pd
# import numpy as np
# import re
# import random
# import string

# # --- Load dataset ---
# df_trans = df_trans.sort_values("Book date")

# # --- Step 1: Sample 500 original rows ---
# original_sample = df_trans.sample(n=500, random_state=42, replace=True).copy()

# # --- Step 2: Anonymization functions ---

# four_digits_letra_pattern = re.compile(r'\*{4}[A-Za-z0-9]{4}')
# eleven_digit_pattern = re.compile(r'(?<!\w)\d{11}(?!\w)')
# eight_digit_pattern = re.compile(r'(?<!\w)\d{8}(?!\w)')

# def random_alphanumeric(length=4):
#     return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

# def random_digits(length):
#     return ''.join(random.choices(string.digits, k=length))

# def anonymize_name(name):
#     if not isinstance(name, str):
#         return name
#     name = four_digits_letra_pattern.sub(lambda _: f'****{random_alphanumeric(4)}', name)
#     name = eleven_digit_pattern.sub(lambda _: random_digits(11), name)
#     name = eight_digit_pattern.sub(lambda _: random_digits(8), name)
#     return name

# # --- Step 3: Create anonymized duplicate ---
# duplicate_sample = original_sample.copy()
# duplicate_sample["Counter party name"] = duplicate_sample["Counter party name"].apply(anonymize_name)

# # --- Step 4: Combine original + duplicate ---
# final_df = pd.concat([original_sample, duplicate_sample], ignore_index=True)

# # --- Step 5: Sort by Book date ---
# final_df = final_df.sort_values("Book date").reset_index(drop=True)

# display(final_df)

In [4]:
import pandas as pd
import numpy as np
import re
import random
import string

# --- Load original dataset ---
df_trans = df_trans.sort_values("Book date")

# --- Step 1: Sample 100 original rows ---
original_sample = df_trans.sample(n=200, random_state=42, replace=True).copy()
original_sample["Source"] = "original"

# --- Step 2: Define anonymization + randomization ---
four_digits_letra_pattern = re.compile(r'\*{4}[A-Za-z0-9]{4}')
eleven_digit_pattern = re.compile(r'(?<!\w)\d{11}(?!\w)')
eight_digit_pattern = re.compile(r'(?<!\w)\d{8}(?!\w)')

def random_alphanumeric(length=4):
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=length))

def random_digits(length):
    return ''.join(random.choices(string.digits, k=length))

def anonymize_name(name):
    if not isinstance(name, str):
        return name
    name = four_digits_letra_pattern.sub(lambda _: f'****{random_alphanumeric(4)}', name)
    name = eleven_digit_pattern.sub(lambda _: random_digits(11), name)
    name = eight_digit_pattern.sub(lambda _: random_digits(8), name)
    return name

def randomize_amount(amount):
    if pd.isna(amount) or not isinstance(amount, (int, float)):
        return amount
    factor = random.uniform(0.5, 1.5)
    return round(amount * factor, 2)

def randomize_credit_debit(value):
    return random.choice(["CREDIT", "DEBIT"])

# --- Step 3: Generate 10x synthetic copies ---
synthetic_samples = []

for i in range(5):  # x10
    synth = original_sample.copy()
    synth["Counter party name"] = synth["Counter party name"].apply(anonymize_name)
    synth["Amount"] = synth["Amount"].apply(randomize_amount)
    
    if "Credit/Debit Indicator" in synth.columns:
        synth["Credit/Debit Indicator"] = synth["Credit/Debit Indicator"].apply(randomize_credit_debit)
    
    synth["Source"] = f"synthetic_{i+1}"
    synthetic_samples.append(synth)

# --- Step 4: Combine original + synthetic ---
combined_df = pd.concat([original_sample] + synthetic_samples, ignore_index=True)

# --- Step 5: Sort by Book date ---
combined_df = combined_df.sort_values("Book date").reset_index(drop=True)

# Book date must be in datetime format
combined_df["Book date"] = pd.to_datetime(combined_df["Book date"], errors='coerce')

# --- Final output ---
print(f"Final DataFrame shape: {combined_df.shape}")  # Should be (1100, ...)
display(combined_df)

# Save the final DataFrame to a CSV file
output_file = '../app/anonymized_transactions(1).csv'
combined_df.to_csv(output_file, index=False)

Final DataFrame shape: (1200, 7)


,Counter party account number,Counter party name,Currency,Amount,Credit/debit indicator,Book date,Source
0,NaN,Received from Other Bank,PHP,951.64,Credit,2025-05-20,synthetic_3
1,NaN,Received from Other Bank,PHP,2029.19,Credit,2025-05-20,synthetic_4
2,NaN,W/D FR SAV BDO/005542 54SM STAMESA BDO,PHP,1932.20,Debit,2025-05-20,synthetic_4
3,NaN,W/D FR SAV BDO/005542 54SM STAMESA BDO,PHP,1291.64,Debit,2025-05-20,synthetic_4
4,NaN,Received from Other Bank,PHP,1013.97,Credit,2025-05-20,synthetic_2
...,...,...,...,...,...,...,...
1195,NaN,Received from BDO Account,PHP,969.15,Credit,2025-06-13,synthetic_2
1196,NaN,Received from BDO Account,PHP,834.35,Credit,2025-06-13,synthetic_2
1197,NaN,POS W/D SV NATIONAL B POS,PHP,295.02,Debit,2025-06-13,synthetic_4
1198,NaN,Sent via InstaPay GXI ****71P9,PHP,516.64,Debit,2025-06-13,synthetic_2
